In [1]:
import pandas as pd
import numpy as np
import datetime

df = pd.read_csv('WALM.csv')

# Converting each date in the Date column to a date object
df['Date'] = pd.to_datetime(df['Date'],format = '%b %d, %Y')
df.dtypes

Date                     datetime64[ns]
Close Price                     float64
Open Price                      float64
High Price                      float64
Low Price                       float64
Total Traded Quantity            object
dtype: object

In [2]:
# Assuming the latest date to be the current date
cur_date = df['Date'].max()

# Calculating the date of 90 days before
l_date = cur_date - datetime.timedelta(days=90)

filt = (df['Date'] > l_date) & (df['Date'] <= cur_date)

type(df.loc[filt,'Close Price'].tail(1))


pandas.core.series.Series

In [4]:
# Calculating Maximum, Minimum and Mean Closing price of the last 90 days
mean_price = df.loc[filt]['Close Price'].mean()
max_price = df.loc[filt]['Close Price'].max()
min_price = df.loc[filt]['Close Price'].min()

print('Walmart')
print('In the last 90 days')
print('Maximum Closing Price: '+str(max_price))
print('Minimum Closing Price: '+str(min_price))
print('Mean Closing Price:    '+str(mean_price))

Walmart
In the last 90 days
Maximum Closing Price: 120.24
Minimum Closing Price: 105.22
Mean Closing Price:    114.83809523809524


In [4]:
# Subtracting the minimum value of date from the maximum value of date
print(df['Date'].max() - df['Date'].min())

729 days 00:00:00


In [3]:
# Creating new column Month and Year
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Year'] = pd.DatetimeIndex(df['Date']).year

# Grouping data by Year and Month
x = df.groupby(['Year','Month'])

convert = {'K': 3, 'M': 6, 'B': 9, 'T': 12}

# Array that will contain VWAP values with Year and Month
vwap_arr = []

for month,val in x:
    # month[0] is a particular year
    # month[1] is the month in that year
    
    # Creating a separate dataframe that includes close price and volume
    dx = val[['Close Price','Total Traded Quantity']]
    
    # Extracting values of Close Price
    cp = dx.iloc[:,0].values
    
    # Converting numpy array to python list
    cp.tolist()
    
    # Extracting values of Total Traded Quantity
    temp = dx.iloc[:,1].values
    
    # Converting the million notation to actual number
    vol = [int(float(i[:-1])*10**convert[i[-1]]) for i in temp]
    
    products = []
    
    # Multiplying Close Price with Volume
    for price,volume in zip(cp,vol):
        products.append(price*volume)
    
    # Sum of products
    p_sum = sum(products)
    # Sum of volume
    v_sum = sum(vol)
    # VWAP value
    vwap = p_sum/v_sum
    temp_dict = {'Year':month[0],'Month':month[1],'VWAP':vwap}
    
    vwap_arr.append(temp_dict)
    
for i in vwap_arr:
    print(i)

{'Year': 2017, 'Month': 10, 'VWAP': 87.89777095486644}
{'Year': 2017, 'Month': 11, 'VWAP': 95.0641977637074}
{'Year': 2017, 'Month': 12, 'VWAP': 97.7977801762907}
{'Year': 2018, 'Month': 1, 'VWAP': 103.29100177460076}
{'Year': 2018, 'Month': 2, 'VWAP': 96.45872525910096}
{'Year': 2018, 'Month': 3, 'VWAP': 88.1879921875}
{'Year': 2018, 'Month': 4, 'VWAP': 86.77398684751803}
{'Year': 2018, 'Month': 5, 'VWAP': 84.15655304249839}
{'Year': 2018, 'Month': 6, 'VWAP': 84.76151975105758}
{'Year': 2018, 'Month': 7, 'VWAP': 87.06581569327552}
{'Year': 2018, 'Month': 8, 'VWAP': 94.64587231352718}
{'Year': 2018, 'Month': 9, 'VWAP': 95.51509905782311}
{'Year': 2018, 'Month': 10, 'VWAP': 96.99664094626438}
{'Year': 2018, 'Month': 11, 'VWAP': 99.28131773520903}
{'Year': 2018, 'Month': 12, 'VWAP': 91.71356340912224}
{'Year': 2019, 'Month': 1, 'VWAP': 95.72140540713457}
{'Year': 2019, 'Month': 2, 'VWAP': 98.31168235505726}
{'Year': 2019, 'Month': 3, 'VWAP': 98.21823946156572}
{'Year': 2019, 'Month': 4, 

In [4]:
# 1.5 
# Function to calculate the average price over the last N days
def avgClosePrice(n):
    # Calculating the last date
    l_date = cur_date - datetime.timedelta(days=n)
    # Filtering using dates
    filt = (df['Date'] > l_date) & (df['Date'] <= cur_date)
    avg_cp = df.loc[filt]['Close Price'].mean()
    return avg_cp

def percentChange(n):
    l_date = cur_date - datetime.timedelta(days=n)
    
    filt1 = (df['Date'] <= cur_date)
    filt2 = (df['Date'] <= l_date)
    
    cur_cp = float(df.loc[filt1].head(1)['Close Price'])
    last_cp = float(df.loc[filt2].head(1)['Close Price'])
    
    p_change = ((cur_cp - last_cp)/last_cp)*100
    return p_change

def displayPercentChange(p):
    if p > 0:
        print("Profit Percentage : "+str(p))
    else:
        print("Loss Percentage : "+str(p*-1))

def userPrompt_AvgClosePrice():
    n = int(input("Enter the number of days for over which you want to see average Close Price : "))
    avg = avgClosePrice(n)
    print("\nAverage Close Price over the last "+str(n)+" days : "+str(avg))
    
def userPrompt_AvgClosePrice_PercentChange():
    n = int(input("Enter the number of days for over which you want to see average Close Price and Percentage change : "))
    avg = avgClosePrice(n)
    p_change = percentChange(n)
    print("\nAverage Close Price over the last "+str(n)+" days : "+str(avg))
    displayPercentChange(p_change)
    
userPrompt_AvgClosePrice()
print('--------------------------------------------------------------------------')
userPrompt_AvgClosePrice_PercentChange()
print('--------------------------------------------------------------------------')
print('\nLast 1 week')
print('Average Price :   '+str(avgClosePrice(7)))
displayPercentChange(percentChange(7))
print('--------------------------------------------------------------------------')
print('\nLast 2 weeks')
print('Average Price :   '+str(avgClosePrice(14)))
displayPercentChange(percentChange(14))
print('--------------------------------------------------------------------------')
print('\nLast 1 month')
print('Average Price :   '+str(avgClosePrice(31)))
displayPercentChange(percentChange(31))
print('--------------------------------------------------------------------------')
print('\nLast 3 months')
print('Average Price :   '+str(avgClosePrice(92)))
displayPercentChange(percentChange(92))
print('--------------------------------------------------------------------------')
print('\nLast 6 months')
print('Average Price :   '+str(avgClosePrice(183)))
displayPercentChange(percentChange(183))
print('--------------------------------------------------------------------------')
print('\nLast 1 year')
print('Average Price :   '+str(avgClosePrice(365)))
displayPercentChange(percentChange(365))

Enter the number of days for over which you want to see average Close Price : 56

Average Close Price over the last 56 days : 117.6379487179487
--------------------------------------------------------------------------
Enter the number of days for over which you want to see average Close Price and Percentage change : 54

Average Close Price over the last 54 days : 117.72684210526313
Profit Percentage : 4.235953089445116
--------------------------------------------------------------------------

Last 1 week
Average Price :   119.38199999999999
Loss Percentage : 0.6174899866488728
--------------------------------------------------------------------------

Last 2 weeks
Average Price :   119.511
Loss Percentage : 0.42638575369952775
--------------------------------------------------------------------------

Last 1 month
Average Price :   118.66086956521737
Profit Percentage : 1.2582894065635009
--------------------------------------------------------------------------

Last 3 months
Averag

In [5]:
# 1.6
# Calculating percent change in close prices in consecutive days using pct_change()
df['Day_Perc_Change'] = df['Close Price'].pct_change()*100
df.at[0,'Day_Perc_Change']=0
df.head()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity,Month,Year,Day_Perc_Change
0,2019-10-24,119.10,119.18,119.56,118.52,2.92M,10,2019,0.000000
1,2019-10-23,119.35,119.17,119.85,118.69,2.87M,10,2019,0.209908
2,2019-10-22,119.58,119.75,120.17,119.39,2.60M,10,2019,0.192711
3,2019-10-21,119.74,119.30,119.83,118.82,3.47M,10,2019,0.133802
4,2019-10-18,119.14,120.45,120.60,119.13,5.59M,10,2019,-0.501086


In [6]:
# 1.7
# Creating a new column Trend
df['Trend'] = 'null'

# a function that check the value of Day_Perc_Change and returns an appropriate Trend value
def trendSetter(p):
    if p >= -0.5 and p < 0.5:
        return 'Slight or No change'
    elif p >= 0.5 and p < 1:
        return 'Slight positive'
    elif p >= 1 and p < 3:
        return 'Positive'
    elif p >= 3 and p < 7:
        return 'Among top gainers'
    elif p >= 7:
        return 'Bull run'
    elif p >= -1 and p < -0.5:
        return 'Slight negative'
    elif p >= -3 and p < -1:
        return 'Negative'
    elif p >= -7 and p < -3:
        return 'Among top losers'
    elif p < -7:
        return 'Bear drop'
    
for i in df.index:
    df.at[i,'Trend'] = trendSetter(df['Day_Perc_Change'][i])

df.head()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity,Month,Year,Day_Perc_Change,Trend
0,2019-10-24,119.10,119.18,119.56,118.52,2.92M,10,2019,0.000000,Slight or No change
1,2019-10-23,119.35,119.17,119.85,118.69,2.87M,10,2019,0.209908,Slight or No change
2,2019-10-22,119.58,119.75,120.17,119.39,2.60M,10,2019,0.192711,Slight or No change
3,2019-10-21,119.74,119.30,119.83,118.82,3.47M,10,2019,0.133802,Slight or No change
4,2019-10-18,119.14,120.45,120.60,119.13,5.59M,10,2019,-0.501086,Slight negative


In [7]:
# 1.8
# Average and Median values of Total Traded Quantity by Trend
import statistics as st
dz = df.groupby('Trend')
for grp in dz:
    # grp[0] is the Trend group
    print('Trend Group : '+grp[0])
    # Coverting the Million notation to actual numbers
    ttq = [int(float(i[:-1])*10**convert[i[-1]]) for i in grp[1]['Total Traded Quantity']]
    print('Average Total Traded Quantity : '+str(st.mean(ttq)))
    print('Median Total Traded Quantity  : '+str(st.median(ttq)))
    print('--------------------------------------------------------------------------')

Trend Group : Among top gainers
Average Total Traded Quantity : 8673333.333333334
Median Total Traded Quantity  : 9165000.0
--------------------------------------------------------------------------
Trend Group : Among top losers
Average Total Traded Quantity : 7710000
Median Total Traded Quantity  : 7710000.0
--------------------------------------------------------------------------
Trend Group : Bear drop
Average Total Traded Quantity : 8525000
Median Total Traded Quantity  : 8525000.0
--------------------------------------------------------------------------
Trend Group : Bull run
Average Total Traded Quantity : 16059999
Median Total Traded Quantity  : 16059999
--------------------------------------------------------------------------
Trend Group : Negative
Average Total Traded Quantity : 8801126.732394366
Median Total Traded Quantity  : 7200000
--------------------------------------------------------------------------
Trend Group : Positive
Average Total Traded Quantity : 10796376.

In [8]:
# 1.9
# Converting the Million notation to actual numbers

df['Total Traded Quantity'] = [int(float(i[:-1])*10**convert[i[-1]]) for i in df['Total Traded Quantity']]

# Exporting to a csv file
df.to_csv('week2.csv')